In [1]:
#regexp:
# #test([^\n]*)\n([^#][^\n]*)\n
# #test\1\n#test  \2\n

#
# класс узел
#   type
#   парсится     
#   отображается __repr__
#   op
#   args
#   vars
#
# класс утверждения
#   отображение: узел
#   внут. структ. : узел
#     получается из отображения
#
# класс доказанного утверждепия
#   утверждение//rel
#   получение из доказанного
#     подстановкой
#     C1
#     совпадает внут.структ.
#     S5
#     S6
#     S7
#     S8
#     создание аксиомы
#   ... ссылки на предков и потомков
#  
# автоматический поиск доказательства
#   ...
#
#
#
#

In [2]:
#test
x = object()

In [3]:
#test
y=object()

In [4]:
#test
x==y

False

In [71]:
class _or:
    type = 'rel'
    op = 'or'
    nobrace = {'ch','not','eq'}
    def __init__(self,lhs,rhs):
        assert lhs.type=='rel' and rhs.type=='rel'
        self.args = [lhs,rhs]
        self.vars = lhs.vars|rhs.vars
    def print(self,used):
        l = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            l = '('+l+')'
        r = self.args[1].print(used)
        if self.args[1].op not in self.nobrace:
            r = '('+r+')'
        return l+' or '+r
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,oeq):
        return oeq.op=='or' and self.args==oeq.args
class Or:
    def __init__(self,lhs,rhs):
        self.show = _or(lhs.show,rhs.show)
        self.int = _or(lhs.int,rhs.int)
    def __repr__(self):
        return repr(self.show)
class _ch:
    op = 'ch'
    def __init__(self,ch,t):
        self.type = t
        self.ch = ch
        self.vars = {ch}
    def print(self,used):
        return self.ch
    def __repr__(self):
        return self.ch
    def __eq__(self,och):
        return och.op=='ch' and self.ch==och.ch and self.type==och.type
class RelCh:
    def __init__(self,ch):
        self.show = _ch(ch,'rel')
        self.int = _ch(ch,'rel')
    def __repr__(self):
        return repr(self.show)

In [6]:
#test
A = RelCh('A')
A

A

In [7]:
#test
A.show.vars

{'A'}

In [8]:
#test
B = RelCh('B')
Or(A,B)

A or B

In [9]:
#test
C = RelCh('C')
Or(A,Or(B,C))

A or (B or C)

In [67]:
#test
A.int==A.int

True

In [73]:
#test
Or(A,B).int==Or(A,B).int

True

In [74]:
#test
Or(A,B)==Or(B,A)

False

In [76]:
class _not:
    type = 'rel'
    op = 'not'
    nobrace = {'not','ex','an','ch','eq'}
    def __init__(self,arg):
        assert arg.type=='rel'
        self.args = [arg]
        self.vars = arg.vars
    def print(self,used):
        a = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            a = '('+a+')'
        return 'not '+a
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,on):
        return on.op=='not' and self.args==on.args
class Not:
    def __init__(self,arg):
        self.show = _not(arg.show)
        self.int = _not(arg.int)
    def __repr__(self):
        return repr(self.show)

In [11]:
#test
Not(A)

not A

In [12]:
#test
Not(Or(A,B))

not (A or B)

In [13]:
#test
Or(Not(A),B)

not A or B

In [14]:
#test
Not(Not(B))

not not B

In [77]:
#test
Not(A).int==Not(A).int

True

In [15]:
#test
A.show.vars

{'A'}

In [16]:
class _imp:
    type = 'rel'
    op = 'imp'
    nobrace = {'ch','or','not','eq'}
    def __init__(self,lhs,rhs):
        assert lhs.type=='rel' and rhs.type=='rel'
        self.args = [lhs,rhs]
        self.vars = lhs.vars|rhs.vars
    def print(self,used):
        l = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            l = '('+l+')'
        r = self.args[1].print(used)
        if self.args[1].op not in self.nobrace:
            r = '('+r+')'
        return l+' => '+r
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,other):
        return other.op=='imp' and self.args==other.args
class Imp:
    def __init__(self,lhs,rhs):
        self.show = _imp(lhs.show,rhs.show)
        self.int = _or(_not(lhs.int),rhs.int)
    def __repr__(self):
        return repr(self.show)


In [17]:
#test
Imp(A,B).int

not A or B

In [18]:
#test
Imp(A,Or(A,B))

A => A or B

In [19]:
#test
class foo:
    @staticmethod
    def bar(x):
        f = foo()
        f.__x = x
        return f
    def getx(self):
        return self.__x
a = foo.bar(6)
a.getx()

6

In [20]:
from copy import copy
def _subst(n,ch,expr):
    assert ch.op=='ch'
    if n.op=='ch':
        if n.ch==ch.ch:
            assert n.type==ch.type
            return expr
        else:
            return n
    elif isinstance(n,bound):
        return n
    else:
        n1 = copy(n)
        n1.args = []
        n1.vars = set()
        for x in n.args:
            y = _subst(x,ch,expr)
            n1.args.append(y)
            n1.vars|=y.vars
        return n1
def subst(a,ch,expr):
    r = object()
    r.show = _subst(a.show,ch,expr)
    r.int = _subst(a.int,ch,expr)

In [21]:
#test
A.show.vars

{'A'}

In [78]:
class bound:
    _default = 'x,y,z,a,b,c,d,e,f,g,h,i,j,k,l,m,n'.split(',')
    type = 'term'
    op = 'bound'
    vars = set()
    def __init__(self,rec):
        self.recommended = rec
    def choose(self,used):
        for ch in self.recommended:
            if ch not in used:
                self.choosen = ch
                return
        for ch in self._default:
            if ch not in used:
                self.choosen = ch
                return
        raise Error("bound can't choose")
    def print(self,used):
        return self.choosen
    def __eq__(self,other):
        return self is other or other is self.cureq
class TermCh:
    def __init__(self,ch):
        self.show = _ch(ch,'term')
        self.int = _ch(ch,'term')
    def __repr__(self):
        return repr(self.show)    
class _tau:
    type = 'term'
    op = 'tau'
    def __init__(self,ch,R,rec = []):
        assert R.type=='rel'
        assert isinstance(ch,_ch),type(ch)
        assert ch.type=='term',ch
        self.bound = bound([ch.ch]+rec)
        self.args = [_subst(R,ch,self.bound)]
        self.vars = self.args[0].vars
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound.choosen}
        return 'tau('+self.bound.print(u)+','+\
                self.args[0].print(u)+')'
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,other):
        if other.op!='tau': return False
        self.bound.cureq = other.bound
        other.bound.cureq = self
        try:
            rez = self.args==other.args
        finally:
            self.bound.cureq = None
            other.bound.cureq = None
        return rez
class Tau:
    def __init__(self,ch,R,rec = []):
        if isinstance(ch,TermCh):
            ch = ch.show
        
        self.show = _tau(ch,R.show,rec)
        self.int = _tau(ch,R.int,rec)
    def __repr__(self):
        return repr(self.show)

In [82]:
#test
x = TermCh('x')
y = TermCh('y')
x

x

In [83]:
#test
t = Tau(x,A)

In [84]:
#test
t = Tau(y,A)

In [85]:
#test
t

tau(y,A)

In [86]:
#test
Tau(x,A).int==Tau(y,A).int

True

In [87]:
#test
Tau(x,A).int==Tau(x,B).int

False

In [50]:
class _eq:
    type = 'rel'
    op = 'eq'
    nobrace = {'ch','tau','bound'}
    def __init__(self,lhs,rhs):
        assert lhs.type=='term' and rhs.type=='term'
        self.args = [lhs,rhs]
        self.vars = lhs.vars|rhs.vars
    def print(self,used):
        l = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            l = '('+l+')'
        r = self.args[1].print(used)
        if self.args[1].op not in self.nobrace:
            r = '('+r+')'
        return l+'=='+r
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,other):
        return other.op=='eq' and self.args==other.args
class Eq:
    def __init__(self,lhs,rhs):
        self.show = _eq(lhs.show,rhs.show)
        self.int = _eq(lhs.int,rhs.int)
    def __repr__(self):
        return repr(self.show)


In [51]:
#test
Eq(x,y)

x==y

In [52]:
#test
Tau(x,Eq(x,y))

tau(x,x==y)

In [53]:
#test
Eq(x,Tau(x,Eq(x,y)))

x==tau(z,z==y)

In [88]:
class _ex:
    type = 'rel'
    prio = 6
    op = 'ex'
    def __init__(self,ch,R,rec = []):
        assert R.type=='rel'
        assert isinstance(ch,_ch),type(ch)
        assert ch.type=='term',ch
        self.bound = bound([ch.ch]+rec)
        self.args = [_subst(R,ch,self.bound)]
        self.vars = self.args[0].vars
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound.choosen}
        return 'Ex '+self.bound.print(u)+':'+\
                self.args[0].print(u)
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,other):
        if other.op!='ex': return False
        self.bound.cureq = other.bound
        other.bound.cureq = self
        try:
            rez = self.args==other.args
        finally:
            self.bound.cureq = None
            other.bound.cureq = None
        return rez
class Ex:
    def __init__(self,ch,R,rec = []):
        if isinstance(ch,TermCh):
            ch = ch.show
        
        self.show = _ex(ch,R.show,rec)
        self.int = _subst(R.int,ch,_tau(ch,R.int,rec))
    def __repr__(self):
        return repr(self.show)

In [55]:
#test
Or(Ex(x,Eq(x,y)),Eq(x,x))

(Ex z:z==y) or x==x

In [56]:
#test
Ex(x,Eq(x,y)).int

tau(x,x==y)==y

In [89]:
class _an:
    type = 'rel'
    prio = 6
    op = 'an'
    def __init__(self,ch,R,rec = []):
        assert R.type=='rel'
        assert isinstance(ch,_ch),type(ch)
        assert ch.type=='term',ch
        self.bound = bound([ch.ch]+rec)
        self.args = [_subst(R,ch,self.bound)]
        self.vars = self.args[0].vars
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound.choosen}
        return 'An '+self.bound.print(u)+':'+\
                self.args[0].print(u)
    def __repr__(self):
        return self.print(self.vars)
    def __eq__(self,other):
        if other.op!='an': return False
        self.bound.cureq = other.bound
        other.bound.cureq = self
        try:
            rez = self.args==other.args
        finally:
            self.bound.cureq = None
            other.bound.cureq = None
        return rez
class An:
    def __init__(self,ch,R,rec = []):
        if isinstance(ch,TermCh):
            ch = ch.show
        
        self.show = _an(ch,R.show,rec)
        self.int = _not(Ex(ch,Not(R),rec).int)
    def __repr__(self):
        return repr(self.show)

In [58]:
#test
An(x,Eq(x,y))

An x:x==y

In [64]:
#test
An(x,Eq(x,y)).int==An(x,Eq(x,y)).int

False

In [65]:
#test
Eq(x,y).int==Eq(x,y).int

False

In [59]:
#test
An(x,Eq(x,y)).int

not not tau(x,not x==y)==y

In [61]:
#test
An(x,A).int

not not A

In [91]:
# класс доказанного утверждепия
#   утверждение//rel
#   получение из доказанного
#     подстановкой
#     C1
#     совпадает внут.структ.
#     S5
#     S6
#     S7
#     S8
#     создание аксиомы
#   ... ссылки на предков и потомков
class Prove:
    @staticmethod
    def subst(p,ch,expr):
        assert isinstance(p,Prove)
        ps = Prove()
        ps.__rel = subst(copy(p.__rel),ch,expr)
        return ps
    @staticmethod
    def C1(_from,_because,_to):
        assert isinstance(_from,Prove) and isinstance(_because,Prove)
        _from = _from.__rel.int
        _beacuse = _beacause.__rel.int
        assert _because.op=='or' and _because.args[0].op=='not'
        assert _because.args[0].args[0]==_from
        assert _because.args[1]==_to.int
        ps = Prove()
        ps.__rel = _to
        return ps
    @staticmethod
    def same(_from,_to):
        assert isinstance(_from,Prove)
        assert _from.__rel.int==_to.int
        ps = Prove()
        ps.__rel = _to
        return ps
    @staticmethod
    def S1(A=None):
        if A==None:
            A = RelCh('A')
        ps = Prove()
        ps.__rel = Imp(Or(A,A),A)
        return ps
    @staticmethod
    def S2(A=None,B=None):
        if B==None:
            assert A==None
            A = RelCh('A')
            B = RelCh('B')
        ps = Prove()
        ps.__rel = Imp(A,Or(A,B))
        return ps
    @staticmethod
    def S3(A=None,B=None):
        if B==None:
            assert A==None
            A = RelCh('A')
            B = RelCh('B')
        ps = Prove()
        ps.__rel = Imp(Or(A,B),Or(B,A))
        return ps
    @staticmethod
    def S4(A=None,B=None,C=None):
        if C==None:
            assert A==None
            A = RelCh('A')
            B = RelCh('B')
            C = RelCh('C')
        ps = Prove()
        ps.__rel = Imp(Imp(A,B),Imp(Or(C,A),Or(C,B)))
        return ps
    @staticmethod
    def S5(R,x,T):
        ps = Prove()
        ps.__rel = Imp(subst(R,x,T),Ex(x,R))
        return ps
    @staticmethod
    def S6(R,x,T,U):
        ps = Prove()
        ps.__rel = Imp(Eq(T,U),Equiv(subst(R,x,T),subst(R,x,U)))
        return ps
    @staticmethod
    def S7(R,S,x):
        ps = Prove()
        ps.__rel = Imp(An(x,Equiv(R,S)),Eq(Tau(x,R),Tau(x,S)))
        return ps
        